## Chapter 9: CRUD Operations with EF Core

With Entity Framework Core (EF Core) set up in your application, it's time to master the core data operations: **Create, Read, Update, and Delete** (CRUD). In this chapter, you'll learn how to perform these operations using EF Core's LINQ-based API, how to efficiently load related data, and how to structure your data access code using the Repository and Unit of Work patterns. By the end, you'll be able to build a robust data layer that is both performant and maintainable.

### 9.1 Querying Data with LINQ

EF Core translates LINQ queries into SQL and executes them against the database. LINQ (Language Integrated Query) provides a powerful, type-safe way to express queries directly in C#.

#### Basic Queries

To retrieve all products:

```csharp
var products = await _context.Products.ToListAsync();
```

To retrieve a single product by ID:

```csharp
var product = await _context.Products.FindAsync(id);
```

`FindAsync` first checks if the entity is already tracked by the context; if not, it queries the database. This is efficient for primary key lookups.

#### Filtering with `Where`

```csharp
var availableProducts = await _context.Products
    .Where(p => p.IsAvailable)
    .ToListAsync();
```

#### Sorting with `OrderBy` / `OrderByDescending`

```csharp
var sortedProducts = await _context.Products
    .OrderBy(p => p.Price)
    .ToListAsync();
```

#### Projection with `Select`

Instead of retrieving entire entities, you can project to a custom type or anonymous object. This reduces the amount of data transferred from the database.

```csharp
var productSummaries = await _context.Products
    .Where(p => p.IsAvailable)
    .Select(p => new ProductSummary
    {
        Id = p.Id,
        Name = p.Name,
        Price = p.Price
    })
    .ToListAsync();
```

Or to an anonymous type (useful within a controller method):

```csharp
var summaries = await _context.Products
    .Select(p => new { p.Id, p.Name, p.Price })
    .ToListAsync();
```

#### Pagination with `Skip` and `Take`

For large datasets, always paginate:

```csharp
int pageSize = 10;
int pageNumber = 1;
var pagedProducts = await _context.Products
    .Skip((pageNumber - 1) * pageSize)
    .Take(pageSize)
    .ToListAsync();
```

#### Aggregations

```csharp
var count = await _context.Products.CountAsync();
var maxPrice = await _context.Products.MaxAsync(p => p.Price);
var averagePrice = await _context.Products.AverageAsync(p => p.Price);
```

#### Querying with Related Data

To include related entities (like `Category`), use the `Include` method (eager loading):

```csharp
var productsWithCategory = await _context.Products
    .Include(p => p.Category)
    .ToListAsync();
```

You can chain `ThenInclude` for deeper relationships:

```csharp
var orders = await _context.Orders
    .Include(o => o.Customer)
    .ThenInclude(c => c.Address)
    .ToListAsync();
```

#### Filtering on Related Data

You can combine `Include` with `Where`:

```csharp
var electronics = await _context.Products
    .Include(p => p.Category)
    .Where(p => p.Category.Name == "Electronics")
    .ToListAsync();
```

#### Raw SQL Queries

If you need to execute raw SQL (e.g., for performance reasons or to use database-specific features), you can use `FromSqlRaw`:

```csharp
var products = await _context.Products
    .FromSqlRaw("SELECT * FROM Products WHERE Price > {0}", 100)
    .ToListAsync();
```

Always use parameterized queries to avoid SQL injection.

### 9.2 Inserting, Updating, and Deleting Records

#### Inserting a New Entity

To add a new product:

```csharp
var newProduct = new Product
{
    Name = "Tablet",
    Price = 299.99m,
    CategoryId = 1,
    IsAvailable = true
};

_context.Products.Add(newProduct);
await _context.SaveChangesAsync();
```

After `SaveChangesAsync`, the `Id` property of `newProduct` will be populated with the database-generated value.

#### Inserting Multiple Entities

```csharp
var products = new List<Product>
{
    new Product { Name = "Monitor", Price = 199.99m, CategoryId = 1 },
    new Product { Name = "Printer", Price = 89.99m, CategoryId = 1 }
};

_context.Products.AddRange(products);
await _context.SaveChangesAsync();
```

#### Updating an Entity

There are two common ways to update an entity:

**1. Retrieve, modify, save:**

```csharp
var product = await _context.Products.FindAsync(id);
if (product != null)
{
    product.Price = 349.99m;
    await _context.SaveChangesAsync();
}
```

This is the simplest and works well when you have the entity tracked by the context.

**2. Attach a detached entity and mark as modified:**

If you have an entity that was created outside the context (e.g., from a controller action), you can attach it and set its state to `Modified`. This approach is useful in disconnected scenarios like web APIs.

```csharp
public async Task UpdateProductAsync(Product product)
{
    _context.Entry(product).State = EntityState.Modified;
    await _context.SaveChangesAsync();
}
```

**Caution:** This method updates all properties, even those that haven't changed. For partial updates, you should retrieve the existing entity and apply changes selectively, or use a dedicated DTO/view model.

#### Deleting an Entity

**1. Retrieve and remove:**

```csharp
var product = await _context.Products.FindAsync(id);
if (product != null)
{
    _context.Products.Remove(product);
    await _context.SaveChangesAsync();
}
```

**2. Attach and remove (if you only know the ID):**

```csharp
var product = new Product { Id = id };
_context.Products.Attach(product);
_context.Products.Remove(product);
await _context.SaveChangesAsync();
```

This avoids an extra database round-trip but assumes the entity exists. If it doesn't, `SaveChangesAsync` will throw a `DbUpdateConcurrencyException`.

### 9.3 Eager Loading, Explicit Loading, and Lazy Loading (and why to avoid Lazy Loading)

EF Core provides three ways to load related data:

#### Eager Loading

We've already seen eager loading with `Include`. It loads related data in the initial query, resulting in a single (or a few) SQL queries with JOINs. This is the most common and efficient approach when you know you'll need the related data.

```csharp
var product = await _context.Products
    .Include(p => p.Category)
    .FirstOrDefaultAsync(p => p.Id == id);
```

#### Explicit Loading

With explicit loading, you first retrieve the entity, then explicitly load its related data using the `Entry` and `Collection` or `Reference` methods.

```csharp
var product = await _context.Products.FindAsync(id);
await _context.Entry(product)
    .Reference(p => p.Category)
    .LoadAsync();
```

For collections:

```csharp
var category = await _context.Categories.FindAsync(id);
await _context.Entry(category)
    .Collection(c => c.Products)
    .LoadAsync();
```

Explicit loading is useful when you conditionally need related data, or to avoid the N+1 query problem in certain scenarios.

#### Lazy Loading

Lazy loading automatically loads related data when you access a navigation property. To enable it, you must:
1. Install the `Microsoft.EntityFrameworkCore.Proxies` package.
2. Enable it when configuring the `DbContext`:
   ```csharp
   options.UseLazyLoadingProxies().UseSqlServer(connectionString);
   ```
3. Make navigation properties `virtual`.

```csharp
public virtual Category Category { get; set; }
```

Then, when you access `product.Category`, EF Core will execute a query to load it if it hasn't been loaded yet.

**Why to avoid lazy loading in web applications:**

- **Performance pitfalls**: It can easily lead to the N+1 query problem, where accessing navigation properties in a loop triggers many additional database queries.
- **Unpredictable behavior**: Queries are executed at unexpected times, potentially outside the scope of the `DbContext` (if the context is disposed).
- **Over-fetching**: You may inadvertently load large amounts of data without realizing it.

For these reasons, **lazy loading is generally discouraged in web applications**. Use eager loading for data you know you'll need, and explicit loading for conditional scenarios.

### 9.4 Using the Repository and Unit of Work Patterns with EF Core

In Chapter 7, we introduced the Repository pattern. With EF Core, you can implement repositories that encapsulate data access logic. However, many developers debate whether the Repository pattern is necessary when using EF Core, since `DbContext` already acts as a repository and unit of work.

#### The Case for Repositories with EF Core

- **Abstraction**: Repositories hide the details of EF Core, making it easier to swap out the data access technology (though that's rare).
- **Testability**: You can easily mock repository interfaces without needing an in‑memory database.
- **Centralized query logic**: Complex queries can be encapsulated in repository methods.

#### The Case Against Repositories

- **Leaky abstraction**: EF Core's `IQueryable` is powerful; repositories often restrict it, leading to complex workarounds.
- **Extra complexity**: You end up writing boilerplate code that EF Core already provides.
- **Loss of EF Core features**: Features like `Include`, `ThenInclude`, and asynchronous composition become harder to expose.

Many modern applications use a **generic repository** for basic CRUD but expose `IQueryable` for queries, or they skip repositories altogether and use the `DbContext` directly in a service layer.

#### A Pragmatic Repository Implementation

Let's create a generic repository that still allows flexible querying.

```csharp
public interface IRepository<T> where T : class
{
    Task<T> GetByIdAsync(int id);
    Task<IEnumerable<T>> GetAllAsync();
    Task<IEnumerable<T>> FindAsync(Expression<Func<T, bool>> predicate);
    Task AddAsync(T entity);
    void Update(T entity);
    void Delete(T entity);
    Task<bool> SaveChangesAsync();
}
```

Implementation:

```csharp
public class EfCoreRepository<T> : IRepository<T> where T : class
{
    protected readonly AppDbContext _context;
    protected readonly DbSet<T> _dbSet;

    public EfCoreRepository(AppDbContext context)
    {
        _context = context;
        _dbSet = context.Set<T>();
    }

    public async Task<T> GetByIdAsync(int id) => await _dbSet.FindAsync(id);

    public async Task<IEnumerable<T>> GetAllAsync() => await _dbSet.ToListAsync();

    public async Task<IEnumerable<T>> FindAsync(Expression<Func<T, bool>> predicate) 
        => await _dbSet.Where(predicate).ToListAsync();

    public async Task AddAsync(T entity) => await _dbSet.AddAsync(entity);

    public void Update(T entity) => _dbSet.Update(entity);

    public void Delete(T entity) => _dbSet.Remove(entity);

    public async Task<bool> SaveChangesAsync() => await _context.SaveChangesAsync() > 0;
}
```

However, this repository doesn't expose `IQueryable`, which limits query flexibility. A better approach is to include `IQueryable` in the repository:

```csharp
public interface IRepository<T> where T : class
{
    IQueryable<T> Query(); // Returns IQueryable for further composition
    // ... other methods
}

public IQueryable<T> Query() => _dbSet.AsQueryable();
```

Then clients can add `Where`, `Include`, etc.:

```csharp
var products = await _productRepository.Query()
    .Include(p => p.Category)
    .Where(p => p.Price > 100)
    .ToListAsync();
```

#### Unit of Work Pattern

The `DbContext` itself is a Unit of Work—it tracks changes and coordinates saving. If you have multiple repositories, you might want them to share the same `DbContext` instance so that changes across repositories are saved atomically. This is often achieved by injecting the same `DbContext` into all repositories and having a service layer call `SaveChangesAsync` once.

Alternatively, you can create a `UnitOfWork` class that exposes repositories and a `SaveChangesAsync` method:

```csharp
public interface IUnitOfWork : IDisposable
{
    IProductRepository Products { get; }
    ICategoryRepository Categories { get; }
    Task<int> CompleteAsync();
}

public class UnitOfWork : IUnitOfWork
{
    private readonly AppDbContext _context;
    public IProductRepository Products { get; }
    public ICategoryRepository Categories { get; }

    public UnitOfWork(AppDbContext context, IProductRepository productRepo, ICategoryRepository categoryRepo)
    {
        _context = context;
        Products = productRepo;
        Categories = categoryRepo;
    }

    public async Task<int> CompleteAsync() => await _context.SaveChangesAsync();

    public void Dispose() => _context.Dispose();
}
```

Then register in DI:

```csharp
builder.Services.AddScoped<IUnitOfWork, UnitOfWork>();
builder.Services.AddScoped<IProductRepository, ProductRepository>();
builder.Services.AddScoped<ICategoryRepository, CategoryRepository>();
```

Your controllers can then inject `IUnitOfWork` and coordinate operations across repositories.

#### Industry Trends

Many teams now prefer to **use EF Core directly in a service layer** without an additional repository layer. They rely on `DbContext` as both repository and unit of work. This keeps the code simple and leverages EF Core's full power. Testing is still possible using an in‑memory database or a test container.

For this book, we'll continue with the repository pattern to emphasize separation of concerns, but you should be aware that it's not mandatory. Choose the approach that best fits your application's complexity.

### 9.5 Practical Example: Building a Product Service

Let's build a service layer that uses our repository to encapsulate business logic.

#### Create a Product Service Interface

```csharp
public interface IProductService
{
    Task<IEnumerable<Product>> GetAvailableProductsAsync();
    Task<Product> GetProductDetailsAsync(int id);
    Task CreateProductAsync(Product product);
    Task UpdateProductAsync(Product product);
    Task DeleteProductAsync(int id);
    Task<bool> ProductExistsAsync(int id);
}
```

#### Implement the Service

```csharp
public class ProductService : IProductService
{
    private readonly IProductRepository _productRepository;
    private readonly ICategoryRepository _categoryRepository;

    public ProductService(IProductRepository productRepository, ICategoryRepository categoryRepository)
    {
        _productRepository = productRepository;
        _categoryRepository = categoryRepository;
    }

    public async Task<IEnumerable<Product>> GetAvailableProductsAsync()
    {
        return await _productRepository.Query()
            .Include(p => p.Category)
            .Where(p => p.IsAvailable)
            .OrderBy(p => p.Name)
            .ToListAsync();
    }

    public async Task<Product> GetProductDetailsAsync(int id)
    {
        var product = await _productRepository.Query()
            .Include(p => p.Category)
            .FirstOrDefaultAsync(p => p.Id == id);

        if (product == null)
            throw new KeyNotFoundException($"Product with id {id} not found.");

        return product;
    }

    public async Task CreateProductAsync(Product product)
    {
        // Business rule: Category must exist
        var categoryExists = await _categoryRepository.ExistsAsync(product.CategoryId);
        if (!categoryExists)
            throw new ArgumentException($"Category with id {product.CategoryId} does not exist.");

        await _productRepository.AddAsync(product);
        await _productRepository.SaveChangesAsync();
    }

    public async Task UpdateProductAsync(Product product)
    {
        var existing = await _productRepository.GetByIdAsync(product.Id);
        if (existing == null)
            throw new KeyNotFoundException($"Product with id {product.Id} not found.");

        // Update allowed fields
        existing.Name = product.Name;
        existing.Price = product.Price;
        existing.Description = product.Description;
        existing.CategoryId = product.CategoryId;
        existing.IsAvailable = product.IsAvailable;

        _productRepository.Update(existing);
        await _productRepository.SaveChangesAsync();
    }

    public async Task DeleteProductAsync(int id)
    {
        var product = await _productRepository.GetByIdAsync(id);
        if (product != null)
        {
            _productRepository.Delete(product);
            await _productRepository.SaveChangesAsync();
        }
    }

    public async Task<bool> ProductExistsAsync(int id) => await _productRepository.ExistsAsync(id);
}
```

#### Refactor Controllers to Use the Service

Now our `ProductsController` can depend on `IProductService` instead of the repository directly. This adds another layer of abstraction but keeps controllers thin and focused on HTTP concerns.

```csharp
public class ProductsController : Controller
{
    private readonly IProductService _productService;

    public ProductsController(IProductService productService)
    {
        _productService = productService;
    }

    public async Task<IActionResult> Index()
    {
        var products = await _productService.GetAvailableProductsAsync();
        return View(products);
    }

    public async Task<IActionResult> Details(int id)
    {
        try
        {
            var product = await _productService.GetProductDetailsAsync(id);
            return View(product);
        }
        catch (KeyNotFoundException)
        {
            return NotFound();
        }
    }

    public IActionResult Create()
    {
        // Populate categories dropdown (we'll need a service for that)
        return View();
    }

    [HttpPost]
    [ValidateAntiForgeryToken]
    public async Task<IActionResult> Create(Product product)
    {
        if (ModelState.IsValid)
        {
            try
            {
                await _productService.CreateProductAsync(product);
                TempData["SuccessMessage"] = "Product created successfully!";
                return RedirectToAction(nameof(Index));
            }
            catch (ArgumentException ex)
            {
                ModelState.AddModelError(string.Empty, ex.Message);
            }
        }
        // Repopulate categories dropdown
        return View(product);
    }

    // ... other actions
}
```

### 9.6 Handling Transactions

EF Core automatically wraps multiple `SaveChanges` calls in a transaction. However, when you need to perform operations across multiple `DbContext` instances or include non‑EF Core operations, you may need explicit transaction control.

#### Using `DbContext.Database.BeginTransaction`

```csharp
using var transaction = await _context.Database.BeginTransactionAsync();
try
{
    // Perform operations
    _context.Products.Add(newProduct);
    await _context.SaveChangesAsync();

    _context.Categories.Add(newCategory);
    await _context.SaveChangesAsync();

    await transaction.CommitAsync();
}
catch
{
    await transaction.RollbackAsync();
    throw;
}
```

#### Using `IExecutionStrategy` for resilience

EF Core can automatically retry failed operations due to transient errors (e.g., deadlocks). You can configure this in `OnConfiguring` or during `AddDbContext`:

```csharp
options.UseSqlServer(connectionString, sqlOptions =>
{
    sqlOptions.EnableRetryOnFailure(
        maxRetryCount: 5,
        maxRetryDelay: TimeSpan.FromSeconds(30),
        errorNumbersToAdd: null);
});
```

Then use the execution strategy to run your transaction:

```csharp
var strategy = _context.Database.CreateExecutionStrategy();
await strategy.ExecuteAsync(async () =>
{
    using var transaction = await _context.Database.BeginTransactionAsync();
    // operations
    await transaction.CommitAsync();
});
```

### Summary

You've now mastered CRUD operations with Entity Framework Core:

- **Querying** with LINQ, including filtering, sorting, projection, pagination, and eager loading.
- **Inserting, updating, and deleting** entities, with attention to tracked vs. detached entities.
- **Loading related data** using eager, explicit, and (discouraged) lazy loading.
- **Repository and Unit of Work patterns** and how to apply them with EF Core, along with current industry opinions.
- **Service layer** encapsulation to separate business logic from controllers.
- **Transaction management** for atomic operations.

With these skills, you can build a robust data layer that is both performant and maintainable. In the next chapter, **"Web APIs with ASP.NET Core,"** you'll learn how to expose your data and business logic as RESTful APIs, enabling integration with frontend frameworks, mobile apps, and third-party services.